## To DO:

Two ways --
* Get Data - Image Frames and cooresponding Time-stamps
* Annotate for Segmentation
* Convert Binary Segmentation to YOLO format using mask_to_YOLO.py
* Use ultralytics YOLOv8 to fine-tune the model on our custom data-set and track objects
* Get corresponding x,y coordinates with time-stamps

or
* Do all of the above manually using CVAT, use polygon tracker this time. 
* Export in COCO format
* and so on

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

def parse_labelmap(labelmap_path):
    """
    Parses the labelmap.txt file to a dictionary
    """
    label_dict = {}
    with open(labelmap_path, 'r') as file:
        for line in file.readlines():
            if not line.startswith('#'):
                parts = line.strip().split(':')
                label_name = parts[0]
                color_str = parts[1]
                label_dict[color_str] = label_name
    return label_dict

In [2]:
parse_labelmap('tmp/labelmap.txt')

{'61,245,61': 'Aerobridge',
 '61,61,245': 'Aircraft',
 '255,204,51': 'Airport_Tractor',
 '140,120,240': 'Baggage_Trolley',
 '226,177,62': 'Catering_Truck',
 '109,218,153': 'Conveyor_Belt',
 '51,221,255': 'Fire_Truck',
 '50,183,250': 'Fuel_Truck',
 '99,228,223': 'GPU',
 '255,255,0': 'Ground_Crew',
 '102,255,102': 'Passengers',
 '241,45,6': 'Pushback_Tug',
 '250,0,0': 'Safety_Cone',
 '0,0,0': 'background'}

In [23]:
def convert_mask_to_YOLO(args):
    '''
    Converts Segmentation masks 1.1 obtained from CVAT to YOLO v8 format
    
    Args:
        input_dir: str, location of segmented masks  '.../masks'
        output_dir: str, output location             '.../labels'
    '''
    input_dir = args.input_dir
    output_dir = args.output_dir

    for j in os.listdir(input_dir):
        image_path = os.path.join(input_dir, j)
        # load the binary mask and get its contours
        mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

        H, W = mask.shape
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # convert the contours to polygons
        polygons = []
        for cnt in contours:
            if cv2.contourArea(cnt) > 10:
                polygon = []
                for point in cnt:
                    x, y = point[0]
                    polygon.append(x / W)
                    polygon.append(y / H)
                polygons.append(polygon)

        # print the polygons
        with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
            for polygon in polygons:
                for p_, p in enumerate(polygon):
                    if p_ == len(polygon) - 1:
                        f.write('{}\n'.format(p))
                    elif p_ == 0:
                        f.write('0 {} '.format(p))
                    else:
                        f.write('{} '.format(p))

            f.close()

In [33]:
label_dict = parse_labelmap('tmp/labelmap.txt')

for j in os.listdir('tmp/SegmentationObject/'):
    image_path = os.path.join('tmp/SegmentationObject/', j)
    color_mask = cv2.imread(image_path, cv2.IMREAD_COLOR)
    gray_mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary_mask = cv2.threshold(gray_mask, 1, 255, cv2.THRESH_BINARY)

    H, W = binary_mask.shape
    contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # write the polygons with class ID
    with open('{}.txt'.format(os.path.join('tmp/labels', j)[:-4]), 'w') as f:
        for cnt in contours:
            if cv2.contourArea(cnt) > 10:
                x,y = cnt[0][0]
                color = tuple(color_mask[y, x].tolist())  #TODO: [x,y]??
                color_str = ','.join(map(str,color))
                class_id = label_dict.get(color_str, 0)

                polygon = []
                for point in cnt:
                    x, y = point[0]
                    polygon.append(x/W)
                    polygon.append(y/H)

                f.write('{}'.format(class_id))
                for p_, p in enumerate(polygon):
                    if p_ == len(polygon) - 1:
                        f.write(' {}\n'.format(p))
                    elif p_ == 0:
                        f.write('class_id {}'.format(p))
                    else:
                        f.write('{} '.format(p))
        
        f.close()
                